# CEC2017 Benchmark: MVO vs Hybrids

Compare MVO, Sequential Hybrid, and Parallel Hybrid on CEC2017 functions

## Setup and Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sys
import os
import io
import contextlib

sys.path.insert(0, os.path.join(os.getcwd(), 'algorithms'))

from cec2017.functions import all_functions
from mvo import MVO
from pso import PSO
from mgwo import MGWO
from sequential_hybrid import sequential_hybrid
from parallel_hybrid import parallel_hybrid
from ga import GA
from gsa import GSA
from abc import ABC

# Create results folder
RESULTS_FOLDER = 'results/cec2017_mvo_hybrids'
os.makedirs(RESULTS_FOLDER, exist_ok=True)

print(f'CEC2017 functions loaded: {len(all_functions)}')
print(f'Results will be saved to: {RESULTS_FOLDER}')

## CEC2017 Wrapper Function

In [ ]:
def cec2017_wrapper(func, name):
    """Wrapper to handle CEC2017 function input format (expects 2D array)"""
    def wrapped(x):
        x_2d = np.array(x).reshape(1, -1)
        return func(x_2d)[0]
    wrapped.__name__ = name
    return wrapped

## Configuration

In [ ]:
DIM = 10           # CEC2017 supports: 10, 30, 50, 100
POP_SIZE = 30
MAX_ITER = 1000
NUM_RUNS = 1       # Increase for statistical analysis
LB = -100          # CEC2017 bounds
UB = 100

print(f'Dimensions: {DIM}')
print(f'Population: {POP_SIZE}')
print(f'Iterations: {MAX_ITER}')
print(f'Runs per function: {NUM_RUNS}')

## Run Benchmark

In [ ]:
CONV_FOLDER = f'{RESULTS_FOLDER}/convergence'
os.makedirs(CONV_FOLDER, exist_ok=True)

colors = {
    'MVO': '#FF0000',        # Bright Red
    'PSO': '#0000FF',        # Bright Blue
    'MGWO': '#00FF00',       # Bright Green
    'Sequential': '#FF00FF',  # Magenta
    'Parallel': '#00FFFF',    # Cyan
    'GA': '#FF8800',         # Orange
    'GSA': '#8800FF',        # Purple
    'ABC': '#FFD700'         # Gold
}
algos_list = ['MVO', 'PSO', 'MGWO', 'Sequential', 'Parallel', 'GA', 'GSA', 'ABC']

# CSV path for incremental saving
results_csv_path = f'{RESULTS_FOLDER}/results_incremental.csv'

solutions_csv_path = f'{RESULTS_FOLDER}/best_solutions.csv'

def save_function_result(func_id, func_results, func_best_solutions):
    """Save single function result and best solutions to CSV immediately after execution"""
    valid_results = {k: v for k, v in func_results.items() if not np.isinf(v)}
    winner = min(valid_results, key=valid_results.get) if valid_results else 'N/A'
    
    # Save fitness results
    row = {
        'Function': f'F{func_id:02d}',
        'MVO': f"{func_results['MVO']:.6e}",
        'PSO': f"{func_results['PSO']:.6e}",
        'MGWO': f"{func_results['MGWO']:.6e}",
        'Sequential': f"{func_results['Sequential']:.6e}",
        'Parallel': f"{func_results['Parallel']:.6e}",
        'GA': f"{func_results['GA']:.6e}",
        'GSA': f"{func_results['GSA']:.6e}",
        'ABC': f"{func_results['ABC']:.6e}",
        'Winner': winner
    }
    
    file_exists = os.path.exists(results_csv_path)
    df_row = pd.DataFrame([row])
    df_row.to_csv(results_csv_path, mode='a', header=not file_exists, index=False)
    
    # Save best solutions (points)
    for algo in algos_list:
        if func_best_solutions[algo] is not None:
            sol_row = {
                'Function': f'F{func_id:02d}',
                'Algorithm': algo,
                'Fitness': f"{func_results[algo]:.6e}",
                'Solution': ','.join([f"{x:.6f}" for x in func_best_solutions[algo]])
            }
            sol_exists = os.path.exists(solutions_csv_path)
            df_sol = pd.DataFrame([sol_row])
            df_sol.to_csv(solutions_csv_path, mode='a', header=not sol_exists, index=False)
    
    print(f"✓ Results saved")


def run_single_function(func_id):
    """Run all algorithms on a single function and plot convergence"""
    
    raw_func = all_functions[func_id - 1]
    func = cec2017_wrapper(raw_func, f"F{func_id:02d}")
    
    print(f"\n{'='*70}")
    print(f"Function F{func_id:02d}")
    print(f"{'='*70}")
    
    func_results = {}
    func_convergence = {}
    
    func_best_solutions = {}
    
    # Run each algorithm
    for algo_name in algos_list:
        print(f"Running {algo_name}...", end=" ", flush=True)
        
        with contextlib.redirect_stdout(io.StringIO()):
            try:
                if algo_name == 'MVO':
                    result = MVO(func, LB, UB, DIM, POP_SIZE, MAX_ITER)
                elif algo_name == 'PSO':
                    result = PSO(func, LB, UB, DIM, POP_SIZE, MAX_ITER)
                elif algo_name == 'MGWO':
                    result = MGWO(func, LB, UB, DIM, POP_SIZE, MAX_ITER)
                elif algo_name == 'Sequential':
                    result = sequential_hybrid(func, LB, UB, DIM, POP_SIZE, MAX_ITER)
                elif algo_name == 'Parallel':
                    result = parallel_hybrid(func, LB, UB, DIM, POP_SIZE, MAX_ITER)
                elif algo_name == 'GA':
                    result = GA(func, LB, UB, DIM, POP_SIZE, MAX_ITER)
                elif algo_name == 'GSA':
                    result = GSA(func, LB, UB, DIM, POP_SIZE, MAX_ITER)
                elif algo_name == 'ABC':
                    result = ABC(func, LB, UB, DIM, POP_SIZE, MAX_ITER)
                else:
                    raise ValueError(f"Unknown algorithm: {algo_name}")
                
                # Get the final best fitness (last value in convergence)
                conv = result.convergence
                fitness = conv[-1]
                func_results[algo_name] = fitness
                func_convergence[algo_name] = conv
                func_best_solutions[algo_name] = result.bestIndividual
            except Exception as e:
                print(f"Error: {e}")
                func_results[algo_name] = float('inf')
                func_convergence[algo_name] = np.full(MAX_ITER, float('inf'))
                func_best_solutions[algo_name] = None
        
        print(f"Done! Fitness: {func_results[algo_name]:.4e}")
    
    # Find winner
    winner = min(func_results, key=func_results.get)
    print(f"\n*** Winner: {winner} (Fitness: {func_results[winner]:.4e}) ***")
    
    # Plot convergence curve (log scale only)
    fig, ax = plt.subplots(figsize=(10, 6))
    iterations = np.arange(1, MAX_ITER + 1)
    
    for algo in algos_list:
        ax.semilogy(iterations, func_convergence[algo], 
                    label=f"{algo} ({func_results[algo]:.2e})", 
                    color=colors[algo], linewidth=2.0, alpha=0.85)
    ax.set_xlabel('Iteration', fontsize=11, fontweight='bold')
    ax.set_ylabel('Fitness (log)', fontsize=11, fontweight='bold')
    ax.set_title(f'CEC2017 F{func_id:02d} - Convergence | Winner: {winner}', fontsize=14, fontweight='bold')
    ax.legend(fontsize=10)
    ax.grid(alpha=0.3, which='both')
    plt.tight_layout()
    
    # Save plot
    save_path = f'{CONV_FOLDER}/F{func_id:02d}_convergence.png'
    plt.savefig(save_path, dpi=200, bbox_inches='tight')
    print(f"Saved: {save_path}")
    plt.show()
    
    return func_results, func_convergence, func_best_solutions

# Run benchmark on all 30 functions
results = {}
convergence = {}
best_solutions = {}

for func_id in range(1, 31):
    func_results, func_convergence, func_best_solutions = run_single_function(func_id)
    results[func_id] = {algo: [func_results[algo]] for algo in algos_list}
    convergence[func_id] = func_convergence
    best_solutions[func_id] = func_best_solutions
    
    # Save immediately after each function
    save_function_result(func_id, func_results, func_best_solutions)


## Results Summary Table

In [ ]:
def create_results_table(results):
    """Create and display results table"""
    
    data = []
    wins = {algo: 0 for algo in algos_list}
    
    # Only process functions that were actually run
    for func_id in sorted(results.keys()):
        means = {algo: np.mean(results[func_id][algo]) for algo in algos_list}
        winner = min(means, key=means.get)
        wins[winner] += 1
        
        row_data = {'Function': f'F{func_id:02d}'}
        for algo in algos_list:
            row_data[algo] = means[algo]
        row_data['Winner'] = winner
        data.append(row_data)
    
    df = pd.DataFrame(data)
    
    csv_path = f'{RESULTS_FOLDER}/final_results.csv'
    df.to_csv(csv_path, index=False, float_format='%.6e')
    print(f"Results saved to: {csv_path}")
    
    return df, wins

df_results, wins = create_results_table(results)
print("\nResults Table:")
print(df_results.to_string(index=False))

## Win Count Summary

In [ ]:
print("\n" + "="*60)
print("WIN COUNT SUMMARY")
print("="*60)
for algo, count in sorted(wins.items(), key=lambda x: x[1], reverse=True):
    bar = "█" * count
    print(f"{algo:15s}: {count:2d} wins {bar}")

## Visualization: Win Count Bar Chart

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))

bar_colors = [colors[a] for a in algos_list]

bars = ax.bar(algos_list, [wins[a] for a in algos_list], color=bar_colors, alpha=0.8, edgecolor='black', linewidth=1.5)

ax.set_ylabel('Number of Wins', fontsize=12, fontweight='bold')
ax.set_xlabel('Algorithm', fontsize=12, fontweight='bold')
ax.set_title(f'CEC2017: Algorithm Comparison Win Count\n({DIM}D, {MAX_ITER} iterations)', fontsize=14, fontweight='bold')
plt.xticks(rotation=45, ha='right')
max_wins = max(wins.values()) if wins.values() else 0
ax.set_ylim(0, max_wins + 2)
ax.grid(alpha=0.3, axis='y')

for bar, algo in zip(bars, algos_list):
    count = wins[algo]
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.3, 
            str(count), ha='center', va='bottom', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig(f'{RESULTS_FOLDER}/win_count.png', dpi=300, bbox_inches='tight')
print(f"Saved: {RESULTS_FOLDER}/win_count.png")
plt.show()

## Visualization: Performance Heatmap

In [ ]:
perf_matrix = df_results[algos_list].values

normalized = np.zeros_like(perf_matrix)
for i in range(perf_matrix.shape[0]):
    row_min = perf_matrix[i].min()
    row_max = perf_matrix[i].max()
    if row_max > row_min:
        normalized[i] = (perf_matrix[i] - row_min) / (row_max - row_min)

fig, ax = plt.subplots(figsize=(12, 14))

im = ax.imshow(normalized, cmap='RdYlGn_r', aspect='auto')

ax.set_xticks(range(len(algos_list)))
ax.set_xticklabels(algos_list, fontsize=10, rotation=45, ha='right')
num_funcs = len(df_results)
ax.set_yticks(range(num_funcs))
ax.set_yticklabels(df_results['Function'].tolist(), fontsize=9)

ax.set_xlabel('Algorithm', fontsize=12, fontweight='bold')
ax.set_ylabel('Function', fontsize=12, fontweight='bold')
ax.set_title('Normalized Performance\n(Green=Best, Red=Worst)', fontsize=14, fontweight='bold')

cbar = plt.colorbar(im, ax=ax, shrink=0.8)
cbar.set_label('Normalized Performance (0=Best, 1=Worst)', fontsize=10)

plt.tight_layout()
plt.savefig(f'{RESULTS_FOLDER}/performance_heatmap.png', dpi=300, bbox_inches='tight')
print(f"Saved: {RESULTS_FOLDER}/performance_heatmap.png")
plt.show()

## Visualization: Category Comparison

In [ ]:
categories = {
    'Unimodal (F1-F3)': range(1, 4),
    'Multimodal (F4-F10)': range(4, 11),
    'Hybrid (F11-F20)': range(11, 21),
    'Composition (F21-F30)': range(21, 31)
}

# Get list of completed function IDs
completed_funcs = set(results.keys())

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

for idx, (cat_name, func_range) in enumerate(categories.items()):
    ax = axes[idx]
    
    cat_wins = {algo: 0 for algo in algos_list}
    funcs_in_cat = [f for f in func_range if f in completed_funcs]
    
    for func_id in funcs_in_cat:
        row = df_results[df_results['Function'] == f'F{func_id:02d}']
        if len(row) > 0:
            winner = row['Winner'].values[0]
            cat_wins[winner] += 1
    
    counts_list = [cat_wins[a] for a in algos_list]
    bar_colors_cat = [colors[a] for a in algos_list]
    
    bars = ax.bar(algos_list, counts_list, color=bar_colors_cat, alpha=0.8, edgecolor='black')
    ax.set_title(f"{cat_name} ({len(funcs_in_cat)} funcs)", fontsize=12, fontweight='bold')
    ax.set_ylabel('Wins')
    max_count = max(counts_list) if counts_list else 0
    ax.set_ylim(0, max(len(funcs_in_cat) if funcs_in_cat else 1, max_count + 1))
    ax.grid(alpha=0.3, axis='y')
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right', fontsize=9)
    
    for bar, algo in zip(bars, algos_list):
        count = cat_wins[algo]
        if count > 0:
            ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
                    str(count), ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.suptitle('Performance by Function Category', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(f'{RESULTS_FOLDER}/category_comparison.png', dpi=300, bbox_inches='tight')
print(f"Saved: {RESULTS_FOLDER}/category_comparison.png")
plt.show()

## Visualization: Box Plot

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

algo_data = {algo: [] for algo in algos_list}
for func_id in results.keys():
    for algo in algo_data.keys():
        algo_data[algo].extend(results[func_id][algo])

log_data = []
labels = []
for algo in algos_list:
    vals = np.array(algo_data[algo])
    vals = np.clip(vals, 1e-10, None)
    log_data.append(np.log10(vals))
    labels.append(algo)

bp = ax.boxplot(log_data, labels=labels, patch_artist=True)

colors_list = [colors[a] for a in algos_list]
for patch, color in zip(bp['boxes'], colors_list):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)

ax.set_ylabel('log10(Fitness)', fontsize=12, fontweight='bold')
ax.set_xlabel('Algorithm', fontsize=12, fontweight='bold')
ax.set_title('Distribution of Fitness Values', fontsize=14, fontweight='bold')
ax.grid(alpha=0.3, axis='y')
plt.xticks(rotation=45, ha='right')

plt.tight_layout()
plt.savefig(f'{RESULTS_FOLDER}/fitness_boxplot.png', dpi=300, bbox_inches='tight')
print(f"Saved: {RESULTS_FOLDER}/fitness_boxplot.png")
plt.show()

## Final Summary

In [ ]:
print("\n" + "="*70)
print("FINAL SUMMARY")
print("="*70)

print(f"\nConfiguration:")
print(f"  Dimensions: {DIM}")
print(f"  Population: {POP_SIZE}")
print(f"  Iterations: {MAX_ITER}")

num_completed = len(results)
print(f"\nOverall Win Count ({num_completed} functions):")
for algo, count in sorted(wins.items(), key=lambda x: x[1], reverse=True):
    pct = count / num_completed * 100 if num_completed > 0 else 0
    print(f"  {algo:15s}: {count:2d}/{num_completed} ({pct:.1f}%)")

print(f"\nAll results saved to: {RESULTS_FOLDER}/")
print("="*70)